In [2]:
from datasets import load_dataset

In [ ]:
data = load_dataset("Synthyra/BIOGRID", split="train")
data


In [ ]:
seqs = set()

data = data.shuffle(seed=42)

def process_dataset(ex):
    seq_a, seq_b = ex['SeqA'], ex['SeqB']
    if seq_a in seqs or seq_b in seqs:
        return False
    seqs.add(seq_a)
    seqs.add(seq_b)
    return True

data = data.filter(process_dataset)
data

In [ ]:
data = load_dataset('Synthyra/all_string_pairs_700', split='train')
data

In [ ]:
class UniqueIDFilter:
    def __init__(self):
        self.ids_seen = set()
    
    def __call__(self, ex):
        if ex['score'] < 900:
            return False
        id_a, id_b = ex['pairs'].split('|')
        if id_a in self.ids_seen or id_b in self.ids_seen:
            return False
        self.ids_seen.add(id_a)
        self.ids_seen.add(id_b)
        return True


filter_func = UniqueIDFilter()
data = data.filter(filter_func, num_proc=16)
data

In [ ]:
data = load_dataset('Synthyra/all_string_pairs_900_unique', split='train')
ids = set()
for pair in data['pairs']:
    id_a, id_b = pair.split('|')
    ids.add(id_a)
    ids.add(id_b)
del data
len(ids)

In [ ]:
string_seqs = load_dataset('Synthyra/StringDBSeqsv12', split='train')


def get_relevant_ids(ex):
    id = ex['id']
    if id in ids:
        return True
    return False

string_seqs = string_seqs.filter(get_relevant_ids)
string_seqs

In [ ]:
string_seqs.push_to_hub('Synthyra/StringDBSeqsv12_unique', private=True)

In [ ]:
seqs = set(load_dataset('Synthyra/StringDBSeqsv12_unique', split='train')['sequence']) # 3% ish in av set
len(seqs)

In [ ]:
seqs = set(load_dataset('lhallee/foldseek_dataset', split='train')['seqs']) # 3% ish in av set
len(seqs)

README.md:   0%|          | 0.00/519 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

train-00000-of-00017.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

train-00001-of-00017.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

...

train-00005-of-00017.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

train-00006-of-00017.parquet:   0%|          | 0.00/414M [00:00<?, ?B/s]

train-00007-of-00017.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

train-00008-of-00017.parquet:   0%|          | 0.00/414M [00:00<?, ?B/s]

train-00009-of-00017.parquet:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [ ]:
from tqdm.auto import tqdm


av = load_dataset('lhallee/AV_large', split='train', streaming=True)

counter, total = 0, 0

pbar = tqdm(av)
for ex in pbar:
    if ex['sequence'] in seqs:
        counter += 1
    total += 1
    pbar.set_description(f'{counter}/{total}')

counter
